<img src="https://i.ibb.co/K5n24Nd/1.png"  alt="KCode Banner"  />

<a href="https://www.linkedin.com/in/kauebr/"><img src="https://cdn-icons-png.flaticon.com/512/174/174857.png" width="30" ></a> <a href="mailto: kauemandarino@gmail.com"><img src="https://cdn-icons-png.flaticon.com/512/5968/5968534.png" width="30" ></a><a href="
https://api.whatsapp.com/send?phone=5541991835800&text=Oi"> <img src="https://imagepng.org/wp-content/uploads/2017/08/whatsapp-icone-1.png" width="30" ></a><a href="https://github.com/kauebr/"><img src="https://github.githubassets.com/images/modules/logos_page/Octocat.png" width="30" ></a>


# Análise exploratório de dados

## Metodologia / Passos:
* Fazer em excel um dataframe com as intereseccoes das datas e puxar `ok`
* Fazer um dataframe com todos os valores das ações das empresas empresas no periodo `ok`
* Tirar a média `ok`
* Calcular a variação  `ok`
* Criar um outro dataframe com a tx Selic `ok`
* Calcular a variação  `ok`
* Fazer a análise exploratória 

In [8]:
# Importando libs
import pandas as pd

# Criando o nossa série temporal

In [9]:
# Puxando os dados
dir = 'C:/Users/manda/OneDrive/Documents/GitHub/Insights/Setor Siderúrgico Nacional/2 Entrega - Cliclo 2/Passo 03 e 04 - Coleta e limpeza dos dados/DataFrames/DF pronto para análise pandas.csv'
df = pd.read_csv(dir, skiprows=1, encoding='UTF8')
# Dropando coluna desnecessária
df = df.drop(columns=['Unnamed: 0'])
df


,Data,Vale,Gerdau Pref,Gerdau Met Pref,CSN,Usiminas,Selic
0,1/31/2000,1.951422,268.362579,0.219444,1.667136,1.731119,1.46
1,2/29/2000,1.658708,263.639343,0.231984,1.566680,1.566250,1.45
2,3/31/2000,1.626185,244.209869,0.200635,1.754768,1.426112,1.45
3,4/28/2000,1.626185,234.012115,0.219444,1.453400,1.244756,1.30
4,5/31/2000,1.700176,1.252358,0.980719,1.709883,1.318947,1.49
...,...,...,...,...,...,...,...
258,3/31/2022,90.889160,29.354248,22.675573,25.608978,13.254291,0.93
259,4/29/2022,79.185761,26.648436,20.724682,20.729202,11.310000,0.83
260,5/31/2022,81.961868,28.365076,22.144821,21.840000,10.960000,1.03
261,6/30/2022,72.787384,21.733269,17.251686,15.440000,8.650000,1.02


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Data             263 non-null    object 
 1   Vale             263 non-null    float64
 2   Gerdau Pref      263 non-null    float64
 3   Gerdau Met Pref  263 non-null    float64
 4   CSN              263 non-null    float64
 5   Usiminas         263 non-null    float64
 6   Selic            263 non-null    float64
dtypes: float64(6), object(1)
memory usage: 14.5+ KB


A data esta no formato str, vamos corrigir:

In [11]:
df['Data']= pd.to_datetime(df['Data'])


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Data             263 non-null    datetime64[ns]
 1   Vale             263 non-null    float64       
 2   Gerdau Pref      263 non-null    float64       
 3   Gerdau Met Pref  263 non-null    float64       
 4   CSN              263 non-null    float64       
 5   Usiminas         263 non-null    float64       
 6   Selic            263 non-null    float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 14.5 KB


# Modelando nossos dados

In [13]:
# Inserindo uma coluna com a média do valor das ações
df['Média ações'] = (df['Vale'] + df['Gerdau Pref'] + df['Gerdau Met Pref'] + df['CSN'] +df['Usiminas'])/5
df

,Data,Vale,Gerdau Pref,Gerdau Met Pref,CSN,Usiminas,Selic,Média ações
0,2000-01-31,1.951422,268.362579,0.219444,1.667136,1.731119,1.46,54.786340
1,2000-02-29,1.658708,263.639343,0.231984,1.566680,1.566250,1.45,53.732593
2,2000-03-31,1.626185,244.209869,0.200635,1.754768,1.426112,1.45,49.843514
3,2000-04-28,1.626185,234.012115,0.219444,1.453400,1.244756,1.30,47.711180
4,2000-05-31,1.700176,1.252358,0.980719,1.709883,1.318947,1.49,1.392417
...,...,...,...,...,...,...,...,...
258,2022-03-31,90.889160,29.354248,22.675573,25.608978,13.254291,0.93,36.356450
259,2022-04-29,79.185761,26.648436,20.724682,20.729202,11.310000,0.83,31.719616
260,2022-05-31,81.961868,28.365076,22.144821,21.840000,10.960000,1.03,33.054353
261,2022-06-30,72.787384,21.733269,17.251686,15.440000,8.650000,1.02,27.172468


## Calculando a variância percentual

In [14]:
# Reduzindo o dataframe
df_filtrado = df[['Data', 'Média ações', 'Selic']]
df_filtrado

,Data,Média ações,Selic
0,2000-01-31,54.786340,1.46
1,2000-02-29,53.732593,1.45
2,2000-03-31,49.843514,1.45
3,2000-04-28,47.711180,1.30
4,2000-05-31,1.392417,1.49
...,...,...,...
258,2022-03-31,36.356450,0.93
259,2022-04-29,31.719616,0.83
260,2022-05-31,33.054353,1.03
261,2022-06-30,27.172468,1.02


In [16]:
df_filtrado.insert(loc = 2, column = 'Δ% Ações', value = round(df_filtrado['Média ações'].pct_change()*100, 2))
df_filtrado.insert(loc = 4, column = 'Δ% Selic', value = round(df_filtrado['Selic'].pct_change()*100, 2))

In [17]:
# Criando um data frame só com as variáveis

df_var = df_filtrado[['Data', 'Δ% Ações', 'Δ% Selic']]

In [47]:
df_var

,Data,Δ% Ações,Δ% Selic
0,2000-01-31,NaN,NaN
1,2000-02-29,-1.92,-0.68
2,2000-03-31,-7.24,0.00
3,2000-04-28,-4.28,-10.34
4,2000-05-31,-97.08,14.62
...,...,...,...
258,2022-03-31,9.36,22.37
259,2022-04-29,-12.75,-10.75
260,2022-05-31,4.21,24.10
261,2022-06-30,-17.79,-0.97


> Finalmente os dados estão modelados e preparados para nossa análise exploratória;

# Análise exploratória dos dados

## Metodologia
* Vou calcular as variações percentuais para os periodos: todo(22a.), 10a, 5a, 3a, 2a, 1a, 6m `ok]`
vou plotar um gráfico com as informações supracitadas

In [99]:
# Definido funções para apresentação visual(str) dos resultados
def linha():
    print(33*'-')
def tit():
    linha()
    print(f"{titulo : ^33}")
    linha()
    print('Rendimento:')

In [104]:
titulo = ('Periodo todo(22a.)')

# Calculando as variações:
acoes = df_var['Δ% Ações'].sum()
selic = df_var['Δ% Selic'].sum()
maior = 'Ações' if max(acoes, selic) == acoes else 'Selic' 
diferenca = max(acoes, selic) - min(acoes, selic)

# Armazendo os resultados em um dicionário 
vp22a = {'Ações' : acoes, 'Selic': selic, 'Maior' : maior, 'diferença': diferenca}

tit()
print(f"Δ% Ações: {acoes : >23}")
print(f"Δ% Selic: {selic : >23}")
linha()
print(f"{maior} renderam  {diferenca}% a mais")
linha()

---------------------------------
       Periodo todo(22a.)        
---------------------------------
Rendimento:
Δ% Ações:                  332.88
Δ% Selic:                  144.32
---------------------------------
Ações renderam  188.56% a mais
---------------------------------


De cara já vemos que que no período total as ações renderam mais que o dobro do que a selic  
Agora vamos ver nos períodos mais proximos

In [40]:
# Últimos dez anos;
periodo = '10 anos'
data_atual = '2022-07-29' 
data_inicial = '2012-07-29'
df_filtrado = df_var[(df_var['Data'] >= data_inicial) & (df_var['Data'] <= data_atual)]
df_filtrado.sum()

C:\Users\manda\AppData\Local\Temp\ipykernel_3632\4090666541.py:8: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_filtrado.sum()


Δ% Ações    166.27
Δ% Selic    143.45
dtype: float64

In [42]:
# Últimos 5 anos;
periodo = '5 anos'
data_atual = '2022-07-29' 
data_inicial = '2017-07-29'
df_filtrado = df_var[(df_var['Data'] >= data_inicial) & (df_var['Data'] <= data_atual)]
df_filtrado.sum()

C:\Users\manda\AppData\Local\Temp\ipykernel_3632\4262090789.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_filtrado.sum()


Δ% Ações    133.49
Δ% Selic     84.08
dtype: float64

In [43]:
# Últimos 3 anos;
periodo = '3 anos'
data_atual = '2022-07-29' 
data_inicial = '2019-07-29'
df_filtrado = df_var[(df_var['Data'] >= data_inicial) & (df_var['Data'] <= data_atual)]
df_filtrado.sum()

C:\Users\manda\AppData\Local\Temp\ipykernel_3632\1939839253.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_filtrado.sum()


Δ% Ações     66.59
Δ% Selic    126.30
dtype: float64

In [44]:
# Últimos 2 anos;
periodo = '2 anos'
data_atual = '2022-07-29' 
data_inicial = '2020-07-29'
df_filtrado = df_var[(df_var['Data'] >= data_inicial) & (df_var['Data'] <= data_atual)]
df_filtrado.sum()

C:\Users\manda\AppData\Local\Temp\ipykernel_3632\1980904673.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_filtrado.sum()


Δ% Ações     59.29
Δ% Selic    192.59
dtype: float64

In [45]:
# Últimos 1 anos;
periodo = '2 anos'
data_atual = '2022-07-29' 
data_inicial = '2021-07-29'
df_filtrado = df_var[(df_var['Data'] >= data_inicial) & (df_var['Data'] <= data_atual)]
df_filtrado.sum()

C:\Users\manda\AppData\Local\Temp\ipykernel_3632\584286412.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_filtrado.sum()


Δ% Ações    -35.01
Δ% Selic    134.83
dtype: float64

In [46]:
# Últimos 6 meses;
periodo = '6 meses'
data_atual = '2022-07-29' 
data_inicial = '2022-01-29'
df_filtrado = df_var[(df_var['Data'] >= data_inicial) & (df_var['Data'] <= data_atual)]
df_filtrado.sum()

C:\Users\manda\AppData\Local\Temp\ipykernel_3632\881468118.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_filtrado.sum()


Δ% Ações   -13.71
Δ% Selic    34.65
dtype: float64

In [ ]:
#COLOCAR AQUI UMA MÉDIA FLUTUANTE
#IMPORTANTE VALIDAR ESSES DADOS 